In [2]:
from auxiliar import *
import torch
import librosa
import soundfile as sf  # This is the correct way to save audio
from pydub import AudioSegment

def frft_angle_manipulation(signal, angles, windows_size, hop_length, complex_part="real_part", window_type_input="rectangular", window_type_output="hann",fs=44100):
    results = []
    for windows_size, hop_length in zip(windows_size, hop_length):
        output = frft_windowed(signal, windows_size, hop_length, angles, complex_part, window_type_input, window_type_output)
        print(f"Angle: {angle}, Window size: {windows_size}, Hop length: {hop_length}")
        # display(Audio(output, rate=fs))
        results.append(output)
    return results

angles = [0.01, 0.05, 0.1]
windows_size = [2048, 2048*2, 2048*4]
hop_length = [1024, 1024*2, 1024*4]

names = ["original", 
         "angle_0.01_ws_2048", "angle_0.01_ws_4096", "angle_0.01_ws_8192",
         "angle_0.05_ws_2048", "angle_0.05_ws_4096", "angle_0.05_ws_8192",
         "angle_0.10_ws_2048", "angle_0.10_ws_4096", "angle_0.10_ws_8192"]

def save_mp3(numpy_array, sample_rate, filename):
    # Convert to 16-bit PCM WAV in-memory
    temp_wav_path = filename.replace(".mp3", ".wav")
    sf.write(temp_wav_path, numpy_array, sample_rate, subtype='PCM_16')

    # Convert to MP3 using pydub
    audio = AudioSegment.from_wav(temp_wav_path)
    audio.export(filename, format="mp3")
    os.remove(temp_wav_path)

for file in os.listdir("sounds/"):
    if file.endswith(".wav"):
        lista_file = []
        print(f"Processing {file}")
        signal = librosa.load(os.path.join("sounds/", file), sr=44100, mono=True)[0]
        signal = torch.tensor(signal[:44100*10])
        fader = torch.cat((torch.ones(44100*9), torch.hann_window(44100*2)[44100:]))
        signal = signal * fader

        for angle in angles:
            output = frft_angle_manipulation(
                signal, angle, windows_size, hop_length,
                complex_part="real_part",
                window_type_input="rectangular",
                window_type_output="hann",
                fs=44100
            )
            lista_file.extend(output)

        os.makedirs("processed", exist_ok=True)
        for i, output in enumerate(lista_file):
            output = output.numpy()
            out_filename = os.path.join("processed", f"{os.path.splitext(file)[0]}_{names[i]}.mp3")
            save_mp3(output, 44100, out_filename)

        print(f"Processed {file}")

Processing chordula.wav
Angle: 0.01, Window size: 2048, Hop length: 1024
Angle: 0.01, Window size: 4096, Hop length: 2048
Angle: 0.01, Window size: 8192, Hop length: 4096
Angle: 0.05, Window size: 2048, Hop length: 1024
Angle: 0.05, Window size: 4096, Hop length: 2048
Angle: 0.05, Window size: 8192, Hop length: 4096
Angle: 0.1, Window size: 2048, Hop length: 1024
Angle: 0.1, Window size: 4096, Hop length: 2048
Angle: 0.1, Window size: 8192, Hop length: 4096
Processed chordula.wav
Processing guitar_rm_2.wav
Angle: 0.01, Window size: 2048, Hop length: 1024
Angle: 0.01, Window size: 4096, Hop length: 2048
Angle: 0.01, Window size: 8192, Hop length: 4096
Angle: 0.05, Window size: 2048, Hop length: 1024
Angle: 0.05, Window size: 4096, Hop length: 2048
Angle: 0.05, Window size: 8192, Hop length: 4096
Angle: 0.1, Window size: 2048, Hop length: 1024
Angle: 0.1, Window size: 4096, Hop length: 2048
Angle: 0.1, Window size: 8192, Hop length: 4096
Processed guitar_rm_2.wav
Processing glasses.wav
A